In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from datetime import datetime
import matplotlib.pyplot as plt
from random import random
import warnings

In [2]:
# Data reading
DATA_FOLDER = "Data/"

epinions = pd.read_table(DATA_FOLDER + 'soc-sign-epinions.txt', 
                         names=['Source','Target','Weight'],comment='#',header=None).rename_axis('Epinions',axis=1)
slashdot = pd.read_table(DATA_FOLDER + 'soc-sign-Slashdot090221.txt', 
                         names=['Source','Target','Weight'],comment='#',header=None).rename_axis('Slashdot',axis=1)
wikielec = pd.read_csv(DATA_FOLDER + 'wikipedia.csv').rename_axis('Wikipedia',axis=1)

In [3]:
# Display of the same structures datasets
display(epinions.head())
display(slashdot.head())
display(wikielec.head())

Epinions,Source,Target,Weight
0,0,1,-1
1,1,128552,-1
2,2,3,1
3,4,5,-1
4,4,155,-1


Slashdot,Source,Target,Weight
0,0,1,1
1,0,2,1
2,0,3,1
3,0,4,1
4,0,5,1


Wikipedia,Source,Target,Weight
0,3,30,1
1,25,30,-1
2,4,30,1
3,5,30,1
4,6,30,1


# PAUL

### Weight average prediction

#### Idea

This prediction method is used to predict edges' weight locally. That means that we do not look at the overall structure of the graph but only at a few nodes. The idea behind this technique is simple. To predict the weight of an edge, we only look at the two nodes forming this edge as well as their connected edges. First, we calculate the average of weights of the outgoing edges for the source node. Then, we also calculate the average of weights of the ingoing edges for the target node. Finally we compare those two means:

- If both are positive, the predicted edge will be positive
- If both are negative, the predicted edge will be negative
- If the two means have different signs, the sign of the largest mean in absolute value will give the edge prediction

For example, if the average of outgoing edges is -0.7 and the average of ingoing edges is 0.3, the predicted edge will then be -1.

###### Why such an algorithm ?
The idea behind this algorithm is that both people's personnality and performance affects the final link between two people. In this algorithm, performance would typically be the average of ingoing weights for the target node and personnality would be the outgoing weights average for the source node. Personnality is important because people do not always rate other people based on an impersonal choice. For example, wikipedia adminiship voters should theoritically vote based on the candidate capability to be administrator and not based on anything else. Performance is important because it reflects how the target appears to other people. For example, on `epinions` dataset, people might be more trustworthy than others and give a general better impression to people. So this algorithm tries to predict edges based on people's performance and personnality and not based on subgraph connections like balance and status theories do.

#### Evaluation method
Before jumping into the algorithms, we will explain the method used to evaluate our results. We are in the case of a binary classifier (+1 and -1) so we need to evaluate both class. We could use a f-score evaluation but the problem is that this method only evaluates the psotive class (precision and recall on positives). So what we will use is the Matthews correlation coefficient which is given by:

$$ MCC=\frac{TP*TN-FP*FN}{\sqrt{(TP+FP)(TP+FN)(TN+FP)(TN+FN)}} $$

This coefficient gives a measure of the quality of a binary classification and will therefore be used to evaluate our predictions. We also give the precision, recall and f-score for each of the binary classes.

In [20]:
def weighted_average_evaluation(G):
    TP = TN = FP = FN = 0
    for e in G.edges:
        if G.edges[e]['Weight'] == G.edges[e]['Predict']:
            if G.edges[e]['Weight'] > 0:
                TP += 1
            else:
                TN += 1
        else:
            if G.edges[e]['Predict'] > 0:
                FP += 1
            else:
                FN += 1
    
    precision_pos = TP/(TP+FP)
    recall_pos    = TP/(TP+FN)
    precision_neg = TN/(TN+FN)
    recall_neg    = TN/(TN+FP)
    F_score_pos = (2*precision_pos*recall_pos/(precision_pos + recall_pos)) if precision_pos + recall_pos != 0 else 0
    F_score_neg = (2*precision_neg*recall_neg/(precision_neg + recall_neg)) if precision_neg + recall_neg != 0 else 0
    
    if TP*TN*FP*FN == 0:
        MCC = 0
    else:
        MCC = (TP*TN - FP*FN)/np.sqrt(float((TP+FP)*(TP+FN)*(TN+FP)*(TN+FP)))
    
    print("TP: %d\nTN: %d\nFP: %d\nFN: %d\n"%(TP, TN, FP, FN))
    print("Precision on positive: %.3f"% precision_pos)
    print("Recall on positive: %.3f"% recall_pos)
    print("Precision on negative: %.3f"% precision_neg)
    print("Recall on negative: %.3f\n"% recall_neg)
    print("F1 score on positive: %.3f"  % F_score_pos)
    print("F1 score on negative: %.3f\n"% F_score_neg)
    print("Accuracy: %.3f"%((TP+TN)/(TP+TN+FP+FN)))
    print("Matthews correlation coefficient: %.3f" %MCC)

#### First algorithm
The first algorithm `average_predict` is the one we have described above. It will first calculate the mean of outgoing weights and ingoing weights for every nodes and store them as attributes `Source score` and `Target score`. Then, for every edges, it will compare the source node score (average outgoing weights) and target node score (average ingoing weights) and predict the sign as explained above. Note that this is not a real case of prediction as we also count the weight of the edge that we want to predict in the averages. This first algorithm is implemented to get a first impression of the results that we could get. Furthermore, it is much faster than the next agorithms so we will use this speed to find some parameters in the next steps.

In [16]:
def average_predict(dataframe):
    # Dataframe to Graph
    G = nx.from_pandas_edgelist(dataframe, source="Source", target="Target", 
                                edge_attr=["Weight"], create_using=nx.DiGraph())
    
    # Set source and target nodes attribute (ingoing and outgoing averages)
    for node in G.nodes:
        out_edges_weight = [G.get_edge_data(*e)['Weight'] for e in G.out_edges(node)]
        in_edges_weight  = [G.get_edge_data(*e)['Weight'] for e in G.in_edges(node)]
        G.nodes[node]["Source score"] = np.mean(out_edges_weight) if len(out_edges_weight) > 0 else np.nan
        G.nodes[node]["Target score"] = np.mean(in_edges_weight)  if len(in_edges_weight)  > 0 else np.nan

    # Compare node attributes for each edges
    for e in G.edges:
        # Retrieve node attributes
        s = G.nodes[e[0]]['Source score']
        t = G.nodes[e[1]]['Target score']
        if np.isnan(s) or np.isnan(t):
            raise ValueError("nan sould not be there")
        
        # Case where source and target weight averages have same signs
        if s*t > 0:
            # Both are positive
            if s > 0:
                G.edges[e]["Predict"] = 1
            # Both are negative
            else:
                G.edges[e]["Predict"] = -1
        
        # Case where source and target weight averages have opposite signs
        elif s*t < 0:
            if s > 0:
                # Get the largest of the two averages (absolute value)
                if s > abs(t):
                    G.edges[e]["Predict"] = 1
                elif s < abs(t):
                    G.edges[e]["Predict"] = -1
                # Both are equal -> we guess
                else:
                    G.edges[e]["Predict"] = (1 if random() < 0.5 else -1)
            else:
                # Get the largest of the two averages (absolute value)
                if abs(s) > t:
                    G.edges[e]["Predict"] = -1
                elif abs(s) < t:
                    G.edges[e]["Predict"] = 1
                # Both are equal -> we guess
                else:
                    G.edges[e]["Predict"] = (1 if random() < 0.5 else -1)
                    
        # Case where one of the two source and target weight averages is 0
        else:
            if s + t > 0:
                G.edges[e]["Predict"] = 1
            elif s + t < 0:
                G.edges[e]["Predict"] = -1
            else:
                G.edges[e]["Predict"] = (1 if random() < 0.5 else -1)
    
    # Evaluation
    weighted_average_evaluation(G)

In [17]:
average_predict(epinions)

TP: 715933
TN: 84634
FP: 39071
FN: 1734

Precision on positive: 0.948
Recall on positive: 0.998
Precision on negative: 0.980
Recall on negative: 0.684

F1 score on positive: 0.972
F1 score on negative: 0.806

Accuracy: 0.952
Matthews correlation coefficient: 0.665


In [18]:
average_predict(slashdot)

TP: 418871
TN: 66632
FP: 57498
FN: 6201

Precision on positive: 0.879
Recall on positive: 0.985
Precision on negative: 0.915
Recall on negative: 0.537

F1 score on positive: 0.929
F1 score on negative: 0.677

Accuracy: 0.884
Matthews correlation coefficient: 0.493


In [19]:
average_predict(wikielec)

TP: 81102
TN: 8714
FP: 13295
FN: 636

Precision on positive: 0.859
Recall on positive: 0.992
Precision on negative: 0.932
Recall on negative: 0.396

F1 score on positive: 0.921
F1 score on negative: 0.556

Accuracy: 0.866
Matthews correlation coefficient: 0.361


#### Discussion
We can see that the accuracies of our predictions for all datasets are already surprisingly good. But. The problem with our datasets is that they contain a lot more positive edges than negative ones. So the average of weights will more often be positve because there are just more positive edges. If we exagerate the idea, it's just as if we put all weights to +1 so that, as the positive weights are dominant, the accuracy will be high. This is why we need a binary class evaluation method to also evaluate the negative weight predictions. This evaluation problem can easily be resolved if we look at the recall on negative evaluations and see that they are low compared to the other evaluations. Having a low recall on negative weights simply means that there are too much false positives which means that too many weights were predicted to be positive as they were in reality negative.

#### Second algorithm

In [21]:
def weighted_average_predict(dataframe, alpha):
    G = nx.from_pandas_edgelist(dataframe, source="Source", target="Target", 
                                edge_attr=["Weight"], create_using=nx.DiGraph())

    for e in G.edges:
        G.edges[e]['Modified Weight'] = (1-alpha) if G.edges[e]['Weight'] > 0 else -alpha

    for node in G.nodes:
        out_edges_weight = [G.get_edge_data(*e)['Modified Weight'] for e in G.out_edges(node)]
        in_edges_weight  = [G.get_edge_data(*e)['Modified Weight'] for e in G.in_edges(node)]
        G.nodes[node]["Source score"] = np.mean(out_edges_weight) if len(out_edges_weight) > 0 else np.nan
        G.nodes[node]["Target score"] = np.mean(in_edges_weight)  if len(in_edges_weight)  > 0 else np.nan


    for e in G.edges:
        s = G.nodes[e[0]]['Source score']
        t = G.nodes[e[1]]['Target score']
        if np.isnan(s) or np.isnan(t):
            raise ValueError("nan cannot exist")

        if s*t > 0:
            if s > 0:
                G.edges[e]["Predict"] = 1
            else:
                G.edges[e]["Predict"] = -1

        elif s*t < 0:
            if s > 0:
                if s > abs(t):
                    G.edges[e]["Predict"] = 1
                elif s < abs(t):
                    G.edges[e]["Predict"] = -1
                else:
                    G.edges[e]["Predict"] = (1 if random() < 0.5 else -1)
            else:
                if abs(s) > t:
                    G.edges[e]["Predict"] = -1
                elif abs(s) < t:
                    G.edges[e]["Predict"] = 1
                else:
                    G.edges[e]["Predict"] = (1 if random() < 0.5 else -1)

        else:
            if s + t > 0:
                G.edges[e]["Predict"] = 1
            elif s + t < 0:
                G.edges[e]["Predict"] = -1
            else:
                G.edges[e]["Predict"] = (1 if random() < 0.5 else -1)

    weighted_average_evaluation(G)

In [6]:
weighted_average_predict(epinions, 0.63)

TP: 704151
TN: 101583
FP: 22122
FN: 13516

Precision on positive: 0.970
Recall on positive: 0.981
Precision on negative: 0.883
Recall on negative: 0.821

F1 score on positive: 0.975
F1 score on negative: 0.851

Accuracy: 0.958
Youden's J statistic: 0.826


In [136]:
weighted_average_predict(slashdot, 0.63)

TP: 397500
TN: 95929
FP: 28201
FN: 27572

Precision on positive: 0.934
Recall on positive: 0.935
Precision on negative: 0.777
Recall on negative: 0.773

F1 score on positive: 0.934
F1 score on negative: 0.775

Accuracy: 0.898
Youden's J statistic: 0.709


0.7092180790991987

In [143]:
weighted_average_predict(wikielec, 0.66)

TP: 76660
TN: 15677
FP: 6332
FN: 5078

Precision on positive: 0.924
Recall on positive: 0.938
Precision on negative: 0.755
Recall on negative: 0.712

F1 score on positive: 0.931
F1 score on negative: 0.733

Accuracy: 0.890
Youden's J statistic: 0.664


weight on personnality/performance

alpha = 0 finds the percentage of positives

In [174]:
# Gradient Descent
delta = np.inf
alpha_prec = 0.5
alpha = 0.55
gamma = 0.6
f = 1
while delta > 1e-4:
    f_prec = weighted_average_predict(epinions, alpha_prec)
    f = weighted_average_predict(epinions, alpha)
    alpha_new = alpha + gamma*(f-f_prec)
    delta = abs(alpha_new - alpha)
    alpha_prec = alpha
    alpha = alpha_new
    print("alpha_new:",alpha_new)
    print("Delta:",delta,"\n")
print(f)

alpha_new: 0.5697640145520472
Delta: 0.019764014552047127 

alpha_new: 0.5740852201160805
Delta: 0.00432120556403337 

alpha_new: 0.5748827121148449
Delta: 0.0007974919987643325 

alpha_new: 0.5750433621949559
Delta: 0.00016065008011101334 

alpha_new: 0.5750853057346298
Delta: 4.1943539673905406e-05 

0.8198333433391702


In [144]:
def lvo_weighted_average_predict(dataframe, alpha):
    G = nx.from_pandas_edgelist(dataframe, source="Source", target="Target", 
                                edge_attr=["Weight"], create_using=nx.DiGraph())
    
    for e in G.edges:
        G.edges[e]['Modified Weight'] = (1-alpha) if G.edges[e]['Weight'] > 0 else -alpha

    for e in G.edges:
        out_edges_weight = [G.get_edge_data(*edge)['Modified Weight'] for edge in G.out_edges(e[0]) if edge != e]
        in_edges_weight  = [G.get_edge_data(*edge)['Modified Weight'] for edge in G.in_edges(e[1])  if edge != e]
        
        len_out = len(out_edges_weight)
        len_in  = len(in_edges_weight)
        
        if 0 == len_out == len_in:
            G.edges[e]["Predict"] = (1 if random() < 0.5 else -1)
        elif len_out == 0:
            t = np.mean(in_edges_weight)
            if t > 0:
                G.edges[e]["Predict"] = 1
            elif t < 0:
                G.edges[e]["Predict"] = -1
            else:
                 G.edges[e]["Predict"] = (1 if random() < 0.5 else -1)
        elif len_in == 0:
            s = np.mean(out_edges_weight)
            if s > 0:
                G.edges[e]["Predict"] = 1
            elif s < 0:
                G.edges[e]["Predict"] = -1
            else:
                 G.edges[e]["Predict"] = (1 if random() < 0.5 else -1)
        else:
            s = np.mean(out_edges_weight)
            t = np.mean(in_edges_weight)
            if s*t > 0:
                if s > 0:
                    G.edges[e]["Predict"] = 1
                else:
                    G.edges[e]["Predict"] = -1

            elif s*t < 0:
                if s > 0:
                    if s > abs(t):
                        G.edges[e]["Predict"] = 1
                    elif s < abs(t):
                        G.edges[e]["Predict"] = -1
                    else:
                        G.edges[e]["Predict"] = (1 if random() < 0.5 else -1)
                else:
                    if abs(s) > t:
                        G.edges[e]["Predict"] = -1
                    elif abs(s) < t:
                        G.edges[e]["Predict"] = 1
                    else:
                        G.edges[e]["Predict"] = (1 if random() < 0.5 else -1)

            else:
                if s + t > 0:
                    G.edges[e]["Predict"] = 1
                elif s + t < 0:
                    G.edges[e]["Predict"] = -1
                else:
                    G.edges[e]["Predict"] = (1 if random() < 0.5 else -1)

    weighted_average_evaluation(G)

In [74]:
lvo_weighted_average_predict(epinions, 0.64)

TP: 690224
TN: 91555
FP: 32150
FN: 27443

Precision on positive: 0.955
Recall on positive: 0.962
Precision on negative: 0.769
Recall on negative: 0.740

F1 score on positive: 0.959
F1 score on negative: 0.754

Accuracy: 0.929
Youden's J statistic: 0.713


0.7130783643955632

In [75]:
lvo_weighted_average_predict(slashdot, 0.64)

TP: 382272
TN: 87730
FP: 36400
FN: 42800

Precision on positive: 0.913
Recall on positive: 0.899
Precision on negative: 0.672
Recall on negative: 0.707

F1 score on positive: 0.906
F1 score on negative: 0.689

Accuracy: 0.856
Youden's J statistic: 0.595


0.5951297599970591

In [96]:
lvo_weighted_average_predict(wikielec, 0.67)

TP: 74961
TN: 15064
FP: 6945
FN: 6777

Precision on positive: 0.915
Recall on positive: 0.917
Precision on negative: 0.690
Recall on negative: 0.684

F1 score on positive: 0.916
F1 score on negative: 0.687

Accuracy: 0.868
Youden's J statistic: 0.603


0.6032168017639983

In [145]:
def wpo_weighted_average_predict(dataframe, alpha, test_percent):
    
    df = dataframe.sample(round(test_percent*dataframe.shape[0])) 

    G = nx.from_pandas_edgelist(dataframe, source="Source", target="Target", 
                                edge_attr=["Weight"], create_using=nx.DiGraph())
    
    G_unknown = nx.from_pandas_edgelist(df, source="Source", target="Target", 
                                edge_attr=None, create_using=nx.DiGraph())

    for e in G.edges:
        if e in G_unknown.edges:
            G.edges[e]['Modified Weight'] = np.nan
        else:
            G.edges[e]['Modified Weight'] = (1-alpha) if G.edges[e]['Weight'] > 0 else -alpha
        
    for node in G_unknown.nodes:
        out_edges_weight = [G.get_edge_data(*e)['Modified Weight'] for e in G.out_edges(node)]
        in_edges_weight  = [G.get_edge_data(*e)['Modified Weight'] for e in G.in_edges(node)]
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            G.nodes[node]["Source score"] = np.nanmean(out_edges_weight)
            G.nodes[node]["Target score"] = np.nanmean(in_edges_weight)
            
    for e in G_unknown.edges:
        s = G.nodes[e[0]]['Source score']
        t = G.nodes[e[1]]['Target score']
        if np.nan == s == t:
            G.edges[e]["Predict"] = (1 if random() < 0.5 else -1)
        elif s == np.nan:
            if t > 0:
                G.edges[e]["Predict"] = 1
            elif t < 0:
                G.edges[e]["Predict"] = -1
            else:
                G.edges[e]["Predict"] = (1 if random() < 0.5 else -1)
        elif t == np.nan:
            if s > 0:
                G.edges[e]["Predict"] = 1
            elif s < 0:
                G.edges[e]["Predict"] = -1
            else:
                G.edges[e]["Predict"] = (1 if random() < 0.5 else -1)
        else:
            if s*t > 0:
                if s > 0:
                    G.edges[e]["Predict"] = 1
                else:
                    G.edges[e]["Predict"] = -1

            elif s*t < 0:
                if s > 0:
                    if s > abs(t):
                        G.edges[e]["Predict"] = 1
                    elif s < abs(t):
                        G.edges[e]["Predict"] = -1
                    else:
                        G.edges[e]["Predict"] = (1 if random() < 0.5 else -1)
                else:
                    if abs(s) > t:
                        G.edges[e]["Predict"] = -1
                    elif abs(s) < t:
                        G.edges[e]["Predict"] = 1
                    else:
                        G.edges[e]["Predict"] = (1 if random() < 0.5 else -1)
            else:
                if s + t > 0:
                    G.edges[e]["Predict"] = 1
                elif s + t < 0:
                    G.edges[e]["Predict"] = -1
                else:
                    G.edges[e]["Predict"] = (1 if random() < 0.5 else -1)

    weighted_average_stats(G)

In [120]:
wpo_weighted_average_predict(epinions, 0.63, 0.15)

TP: 99175
TN: 13566
FP: 4893
FN: 8572

Precision on positive: 0.953
Recall on positive: 0.920
Precision on negative: 0.613
Recall on negative: 0.735

F1 score on positive: 0.936
F1 score on negative: 0.668

Accuracy: 0.893
Youden's J statistic: 0.605


0.6047556212713925

In [119]:
wpo_weighted_average_predict(slashdot, 0.63, 0.15)

TP: 55617
TN: 12430
FP: 6136
FN: 8197

Precision on positive: 0.901
Recall on positive: 0.872
Precision on negative: 0.603
Recall on negative: 0.670

F1 score on positive: 0.886
F1 score on negative: 0.634

Accuracy: 0.826
Youden's J statistic: 0.520


0.5201507080917165

In [118]:
wpo_weighted_average_predict(wikielec, 0.66, 0.15)

TP: 11286
TN: 2183
FP: 1119
FN: 974

Precision on positive: 0.910
Recall on positive: 0.921
Precision on negative: 0.691
Recall on negative: 0.661

F1 score on positive: 0.915
F1 score on negative: 0.676

Accuracy: 0.866
Youden's J statistic: 0.591


0.5910989454070832

Extension: use dates exponential